In [718]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers # type: ignore
from sklearn.preprocessing import MinMaxScaler
import requests
import time
from datetime import datetime, timedelta
import warnings

# Suppress FutureWarning from scikit-learn
warnings.simplefilter(action='ignore', category=FutureWarning)

In [719]:
def save_dataframe_to_csv(df: pd.DataFrame, ticker: str, filename: str = None) -> bool:
    """
    Saves a pandas DataFrame to a CSV file.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        ticker (str): The stock ticker symbol, used for the default filename.
        filename (str, optional): The desired filename. If None, a default
                                  filename is generated (e.g., 'AAPL_combined_data_YYYY-MM-DD.csv').
                                  Defaults to None.

    Returns:
        bool: True if the file was saved successfully, False otherwise.
    """
    if not isinstance(df, pd.DataFrame) or df.empty:
        print("Error: Input is not a valid or non-empty DataFrame. Nothing to save.")
        return False

    if filename is None:
        # Create a dynamic default filename if one isn't provided.
        # This prevents overwriting previous files.
        timestamp = datetime.now().strftime('%Y-%m-%d')
        filename = f"{ticker}_combined_data_{timestamp}.csv"

    try:
        # Save the DataFrame to a CSV file.
        # The index (which is the date) is crucial, so we ensure it's saved.
        df.to_csv(filename, index=True)
        print(f"\nDataFrame successfully saved to '{filename}'")
        return True
    except IOError as e:
        # Handle specific file system errors
        print(f"\nAn I/O error occurred while saving the file: {e}")
        return False
    except Exception as e:
        # Handle any other unexpected errors
        print(f"\nAn unexpected error occurred: {e}")
        return False

In [720]:
api_key = "v0Y7rqjEfz0nBiixKBqJwLLgyFYbOUGA"
def get_historical_data_fmp(symbol, period_in_quarters):
    """
    Fetches historical price data from FMP API.
    Calculates start_date based on period_in_quarters from current date.
    Returns a DataFrame with 'open', 'high', 'low', 'close', 'volume' prices and a DatetimeIndex.
    """
    end_date = datetime.now()
    # Approximate days per quarter: 365.25 / 4 = 91.3125 days. Using 91.3125 for better accuracy.
    start_date = end_date - timedelta(days=period_in_quarters * 91.3125)

    print(f"Fetching historical price data for {symbol} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} from FMP API...")
    
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date.strftime('%Y-%m-%d')}&to={end_date.strftime('%Y-%m-%d')}&apikey={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        data = response.json()

        if 'historical' in data and data['historical']:
            df = pd.DataFrame(data['historical'])
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date').sort_index()
            # Ensure required columns are present and correctly named
            df = df[['open', 'high', 'low', 'close', 'volume']]
            print(f"Successfully fetched {len(df)} historical price records from FMP.")
            return df
        else:
            print(f"FMP API returned no historical price data for {symbol} in the specified period.")
            return pd.DataFrame()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching historical price data from FMP API: {e}")
        return pd.DataFrame()

**Relative Strength Index (RSI)**

**RSI (Relative Strength Index)** is a **momentum indicator** used in technical analysis to measure the **speed and magnitude of recent price changes**. It helps identify **overbought or oversold conditions** in a stock or other asset.

The standard calculation uses 14 periods (usually days):  

RSI = 100 - (100 / (1 + RS)). 

RS = Average Gain over 14 periods / Average Loss over 14 periods. 

In [721]:
# --- Indicator Calculation ---
def calculate_rsi(df, window=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

**MACD (Moving Average Convergence Divergence)**

**MACD** is a popular **trend-following momentum indicator** used in technical analysis. It reveals changes in the strength, direction, momentum, and duration of a stock’s price trend.

***Formula***

Let:

- **EMA₁₂** = 12-period Exponential Moving Average  
- **EMA₂₆** = 26-period Exponential Moving Average. 

MACD Line     = EMA₁₂ - EMA₂₆. 

Signal Line   = 9-period EMA of the MACD Line

In [722]:
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    exp1 = df['close'].ewm(span=short_window, adjust=False).mean()
    exp2 = df['close'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

**Momentum Indicator**

The Momentum Indicator is a simple yet effective technical analysis tool that measures the **rate of change** of a stock's price. It helps identify the strength of a trend and potential reversals.

***Formula***

Momentum = Current Closing Price - Closing Price  n  periods ago

In [723]:
def calculate_momentum(df, window=1):
    df['Momentum'] = df['close'].diff(window)
    return df

**On-Balance Volume (OBV)**

On-Balance Volume (OBV) is a technical analysis indicator that relates volume to price movement. It measures cumulative buying and selling pressure by adding volume on up days and subtracting it on down days.

***OBV Formula***

Let:

- OBVₜ = OBV on day t
- Volumeₜ = trading volume on day t

In [724]:
def calculate_obv(df):
    df['OBV'] = (np.sign(df['close'].diff()) * df['volume']).fillna(0).cumsum()
    return df

**Average True Range (ATR)**

ATR (Average True Range) is a technical analysis indicator that measures market volatility. It tells how much an asset typically moves (in price) during a given time period.

***Formula***

True Range (TR):

The True Range (TR) for a given day is the maximum of the following:  

TR = max(
High_t - Low_t,
|High_t - Close_{t-1}|,
|Low_t - Close_{t-1}|
). 

***Average True Range (ATR):***

ATR is a moving average (commonly 14 periods) of the True Range:  
ATR_t = (1/n) * sum(TR_{t-i}) for i = 0 to n-1

In [725]:
def calculate_atr(df, window=14):
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    # Combine the three True Range components into one DataFrame:
    # 1. high_low: High - Low (intra-day range)
    # 2. high_close: |High - Previous Close| (gap up/down possibility)
    # 3. low_close: |Low - Previous Close| (gap down possibility)
    # Then take the row-wise max to compute the True Range for each day
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.ewm(span=window, adjust=False).mean()
    return df


In [726]:
def get_fundamental_data_fmp(symbol, period_in_quarters):
    """
    Fetches fundamental data (Income Statement, Key Metrics) from FMP API
    and calculates EPS Growth, Revenue Growth, and ROE.
    Returns a DataFrame with fundamental metrics and a DatetimeIndex.
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=period_in_quarters * 91.3125)

    print(f"Fetching fundamental data for {symbol} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} from FMP API...")

    fundamental_df_list = []

    try:
        # Fetch Income Statements (quarterly)
        income_url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=quarter&limit={period_in_quarters * 2}&apikey={api_key}"
        income_response = requests.get(income_url)
        income_response.raise_for_status()
        income_statements = income_response.json()
        
        income_df = pd.DataFrame(income_statements)
        if not income_df.empty:
            income_df['date'] = pd.to_datetime(income_df['date'])
            income_df = income_df.set_index('date').sort_index()
            
            # Calculate Revenue Growth (Year-over-Year Quarterly)
            # Ensure 'revenue' column exists
            if 'revenue' in income_df.columns:
                income_df['Revenue_Growth'] = income_df['revenue'].pct_change(periods=4) # Compare to same quarter last year
            else:
                income_df['Revenue_Growth'] = np.nan
                print("Warning: 'revenue' column not found in income statements for Revenue_Growth calculation.")

            # Calculate EPS Growth (Year-over-Year Quarterly)
            # Ensure 'eps' column exists
            if 'eps' in income_df.columns:
                income_df['EPS_Growth'] = income_df['eps'].pct_change(periods=4) # Compare to same quarter last year
            else:
                income_df['EPS_Growth'] = np.nan
                print("Warning: 'eps' column not found in income statements for EPS_Growth calculation.")
            
            income_df = income_df[['Revenue_Growth', 'EPS_Growth']]
            fundamental_df_list.append(income_df)
        else:
            print("No income statements found.")

        # Fetch Key Metrics (quarterly) for ROE
        key_metrics_url = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?period=quarter&limit={period_in_quarters * 2}&apikey={api_key}"
        key_metrics_response = requests.get(key_metrics_url)
        key_metrics_response.raise_for_status()
        key_metrics = key_metrics_response.json()

        key_metrics_df = pd.DataFrame(key_metrics)
        if not key_metrics_df.empty:
            key_metrics_df['date'] = pd.to_datetime(key_metrics_df['date'])
            key_metrics_df = key_metrics_df.set_index('date').sort_index()
            # Ensure 'roe' column exists
            if 'roe' in key_metrics_df.columns:
                key_metrics_df['ROE'] = key_metrics_df['roe']
            else:
                key_metrics_df['ROE'] = np.nan
                print("Warning: 'roe' column not found in key metrics for ROE.")
            key_metrics_df = key_metrics_df[['ROE']]
            fundamental_df_list.append(key_metrics_df)
        else:
            print("No key metrics found.")

        if not fundamental_df_list:
            print("No fundamental data could be fetched or processed.")
            return pd.DataFrame()

        # Merge all fundamental dataframes
        merged_fundamental_df = pd.concat(fundamental_df_list, axis=1).sort_index()
        # Filter by date range
        merged_fundamental_df = merged_fundamental_df[(merged_fundamental_df.index >= start_date) & (merged_fundamental_df.index <= end_date)]
        merged_fundamental_df.index.name = 'date'
        print(f"Successfully fetched and processed {len(merged_fundamental_df)} fundamental records from FMP.")
        return merged_fundamental_df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching fundamental data from FMP API: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred during fundamental data processing: {e}")
        return pd.DataFrame()

In [ ]:
def prepare_dataframe(symbol, period_in_quarters):
    """
    Prepares a DataFrame with historical price data, calculated technical indicators,
    and merged fundamental data. The date will be set as the DataFrame's index.
    """
    print(f"\n--- Preparing DataFrame for {symbol} for {period_in_quarters} quarters ---")
    # Pass the api_key to the data fetching functions
    price_data = get_historical_data_fmp(symbol, period_in_quarters)
    if price_data.empty:
        print("Error: No historical price data found.")
        return None

    df = price_data.copy()
    print(f"Initial price_data shape: {df.shape}")

    # Calculate technical indicators
    df = calculate_rsi(df)
    df = calculate_macd(df)
    df = calculate_momentum(df)
    df = calculate_obv(df)
    df = calculate_atr(df)
    print(f"DataFrame shape after technical indicator calculation: {df.shape}")

    # Drop NaNs that result from initial technical indicator calculations
    # This is crucial before merging fundamental data to ensure proper alignment.
    df.dropna(inplace=True)
    print(f"DataFrame shape after dropping NaNs from indicator calculation: {df.shape}")
    if df.empty:
        print("Error: DataFrame is empty after calculating indicators and dropping NaNs.")
        return None
    
    # Merge with fundamental data (forward-fill)
    # Pass the api_key to the data fetching functions
    fundamental_df = get_fundamental_data_fmp(symbol, period_in_quarters)
    print(f"Fundamental data shape: {fundamental_df.shape}")
    if fundamental_df.empty:
        print("Warning: No fundamental data generated by FMP API. This might lead to empty fundamental columns.")

    df = df.reset_index().rename(columns={"index": "date"})
    fundamental_df = fundamental_df.reset_index().rename(columns={"index": "date"})
    
    df_merged = pd.merge(df, fundamental_df, on='date', how='outer')    
    df_merged.set_index('date', inplace=True)
    
    print(f"DataFrame shape after left merge with fundamental data: {df_merged.shape}")
    print("NaN count in fundamental columns after merge (before ffill):")
    if not df_merged.empty:
        for col in ['EPS_Growth', 'Revenue_Growth', 'ROE']:
            if col in df_merged.columns:
                print(f"  {col}: {df_merged[col].isna().sum()} NaNs")

    df_filled = df_merged.fillna(method='ffill').dropna()

    save_dataframe_to_csv(df_filled, "cache/" + symbol + "_merged")  # Save the merged DataFrame to CSV
    print(f"DataFrame shape after forward-fill: {df_filled.shape}")
    print("NaN count in fundamental columns after ffill (before final dropna):")
    if not df_filled.empty:
        for col in ['EPS_Growth', 'Revenue_Growth', 'ROE']:
            if col in df_filled.columns:
                print(f"  {col}: {df_filled[col].isna().sum()} NaNs")

    # The final dropna() is crucial for removing any leading NaNs that couldn't be ffilled
    # (e.g., if fundamental data starts much later than price data, or if there are initial NaNs
    # in technical indicators that were not dropped previously).
    df_final = df_filled.dropna()
    print(f"DataFrame shape after dropping remaining NaNs: {df_final.shape}")

    if df_final.empty:
        print("Error: DataFrame is empty after merging, forward-filling, and dropping NaNs. This might be due to insufficient initial fundamental data or too many leading NaNs.")
        return None

    print(f"DataFrame prepared with {len(df_final)} rows and {len(df_final.columns)} columns.")
    print(f"DataFrame index name: {df_final.index.name}")
    return df_final

In [ ]:
def create_training_testing_datasets(df, lookback_window=30, predict_horizon=1):
    """
    Creates training and testing datasets (X, Y) from a prepared DataFrame,
    applying scaling to both inputs and outputs.
    """
    if df is None or df.empty:
        print("Error: Input DataFrame is empty or None.")
        return None, None, None, None

    print(f"\n--- Creating Training/Testing Datasets ---")

    # Select features (inputs) and targets (outputs)
    input_features = [
        'RSI', 'MACD', 'MACD_Signal', 'Momentum', 'OBV', 'ATR',
        'EPS_Growth', 'Revenue_Growth', 'ROE'
    ]
    output_targets = ['RSI', 'MACD', 'close'] # 'close' here refers to the future close price

    # Ensure all required columns exist after calculations and merges
    missing_cols = [col for col in input_features + output_targets if col not in df.columns]
    if missing_cols:
        print(f"Error: Missing required columns in DataFrame: {missing_cols}")
        return None, None, None, None

    # Normalize indicators
    scaler_input = MinMaxScaler()
    scaler_output = MinMaxScaler()

    # Create input and output sequences
    X, Y = [], []
    # The range for `i` must ensure that `i + lookback_window + predict_horizon - 1`
    # does not exceed the length of the DataFrame.
    # The last valid index for `Y` will be `len(df) - 1`.
    # So, `i + lookback_window + predict_horizon - 1 <= len(df) - 1`
    # which simplifies to `i <= len(df) - lookback_window - predict_horizon`.
    for i in range(len(df) - lookback_window - predict_horizon + 1):
        # Input: `lookback_window` days of selected indicators
        X.append(df[input_features].iloc[i : i + lookback_window].values)
        # Output: `predict_horizon` day's RSI, MACD, and 'close' price
        Y.append(df[output_targets].iloc[i + lookback_window + predict_horizon - 1].values)

    X = np.array(X)
    Y = np.array(Y)

    if X.size == 0 or Y.size == 0:
        print(f"Not enough data after windowing with lookback_window={lookback_window} and predict_horizon={predict_horizon}.")
        print("Adjust lookback_window or predict_horizon, or ensure sufficient input data.")
        return None, None, None, None

    # Apply scaling to input features
    # Reshape X to (n_samples * lookback_window, n_features) for scaling
    original_shape_X = X.shape
    X_reshaped_for_scaling = X.reshape(-1, original_shape_X[-1])
    X_scaled_reshaped = scaler_input.fit_transform(X_reshaped_for_scaling)
    X_scaled = X_scaled_reshaped.reshape(original_shape_X)

    # Apply scaling to output targets
    Y_scaled = scaler_output.fit_transform(Y)

    print(f"Created X_scaled with shape: {X_scaled.shape}")
    print(f"Created Y_scaled with shape: {Y_scaled.shape}")

    return X_scaled, Y_scaled, scaler_input, scaler_output

In [729]:
if __name__ == '__main__':
    symbol_to_process = "AAPL" # Example symbol
    desired_period_in_quarters = 80 # For 20 years of data (20 * 4 = 80 quarters)
    # IMPORTANT: Replace "YOUR_FMP_API_KEY" with your actual FMP API key

    # Step 1: Prepare the DataFrame
    # Pass the API key to prepare_dataframe
    prepared_df = prepare_dataframe(symbol_to_process, desired_period_in_quarters)

    if prepared_df is not None:
        print("\n--- First few rows of Prepared DataFrame ---")
        print(prepared_df.head())
        print("\n--- Last few rows of Prepared DataFrame ---")
        print(prepared_df.tail())
        print(f"\nPrepared DataFrame shape: {prepared_df.shape}")

        # Step 2: Create training and testing datasets
        X_scaled, Y_scaled, scaler_input, scaler_output = create_training_testing_datasets(
            prepared_df,
            lookback_window=30,
            predict_horizon=1
        )

        if X_scaled is not None:
            print("\nSuccessfully created scaled datasets and scalers.")
            # You can now proceed with splitting into train/test sets and model training
            # from sklearn.model_selection import train_test_split
            # X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.2, random_state=42)
            # print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
            # print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")


--- Preparing DataFrame for AAPL for 80 quarters ---
Fetching historical price data for AAPL from 2005-07-11 to 2025-07-11 from FMP API...
Successfully fetched 5032 historical price records from FMP.
Initial price_data shape: (5032, 5)
DataFrame shape after technical indicator calculation: (5032, 11)
DataFrame shape after dropping NaNs from indicator calculation: (5019, 11)
Fetching fundamental data for AAPL from 2005-07-11 to 2025-07-11 from FMP API...
Successfully fetched and processed 79 fundamental records from FMP.
Fundamental data shape: (79, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            79 non-null     datetime64[ns]
 1   Revenue_Growth  79 non-null     float64       
 2   EPS_Growth      79 non-null     float64       
 3   ROE             79 non-null     float64       
dtypes: datetime64[ns](1), 